# Análisis Fundamental del S&P 500
Este archivo reúne utilidades, cálculos y visualizaciones que permiten comprender de forma exploratoria la salud financiera, la rentabilidad y el costo del capital de empresas.
Se definen helpers para limpiar datos, estimar tasas como el WACC o ROIC y leer series históricas, luego se combinan con símbolos de mercado para mostrar ratios clave, escenarios de flujo, y comparaciones sectoriales.
El foco está en facilitar la comparacion de forma rápida entre distintos tickers.


Se recomienda correr el programa desde Visual Studio Code para poder acceder al dashboard intereactivo.

In [19]:
import pandas as pd
import math
from datetime import datetime as _dt, timedelta as _td
from functools import lru_cache
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import yfinance as yf
from functools import lru_cache
from IPython.display import display



La celda siguiente define ayudantes para convertir y normalizar series numéricas, manejar valores no finitos y aplicar límites seguros antes de usarlos dentro de los cálculos de ratios financieros como WACC y ROIC.

In [20]:
def _to_numeric_series(s):
    """Devuelve la serie como float, coercionando strings/objetos a NaN cuando no sean numéricos."""
    if s is None:
        return pd.Series(dtype=float)
    try:
        return pd.to_numeric(pd.Series(s), errors="coerce")
    except Exception:
        try:
            # Si ya es una Series, coercionar directamente
            return pd.to_numeric(s, errors="coerce")
        except Exception:
            return pd.Series(dtype=float)
        

def _first_valid_float(s):
    """Primero válido y finito como float, o None."""
    try:
        s = _to_numeric_series(s).dropna()
        for v in s:
            if np.isfinite(v):
                return float(v)
        return None
    except Exception:
        return None
    
def _as_float(x):
    """Convierte cualquier valor a float finito si es posible; caso contrario devuelve np.nan."""
    try:
        v = float(x)
        return v if np.isfinite(v) else np.nan
    except Exception:
        # remover separadores comunes
        try:
            v = float(str(x).replace(",", "").replace(" ", ""))
            return v if np.isfinite(v) else np.nan
        except Exception:
            return np.nan
        

def _finite_or_none(x):
    """Devuelve float(x) si es finito; si no, None."""
    v = _as_float(x)
    return None if (isinstance(v, float) and not np.isfinite(v)) or pd.isna(v) else v

def _bound(x, lo=None, hi=None):
    """Acota x entre [lo, hi] si corresponde."""
    v = _as_float(x)
    if pd.isna(v):
        return np.nan
    if lo is not None:
        v = max(lo, v)
    if hi is not None:
        v = min(hi, v)
    return v

In [21]:
# ========= Helpers WACC (con chequeos) =========
def _latest_non_na(series):
    try:
        s = _to_numeric_series(series).dropna()
        return float(s.iloc[0]) if len(s) else None
    except Exception:
        return None


def _fetch_risk_free_rate():
    try:
        tnx = yf.Ticker("^TNX").history(period="5d", interval="1d")
        if tnx is None or tnx.empty:
            return None
        val = _as_float(tnx["Close"].iloc[-1]) / 100.0
        return _finite_or_none(val)
    except Exception:
        return None


def _compute_tax_rate(income):
    if income is None or getattr(income, "empty", True):
        return None
    tax_names = ["Income Tax Expense","Provision for Income Taxes","Tax Provision","Income Tax Expense Benefit"]
    ebt_names = ["Income Before Tax","Earnings Before Tax","Pretax Income","Income Before Tax (EBT)"]
    tax = ebt = None
    for n in tax_names:
        if n in income.index:
            tax = _latest_non_na(income.loc[n])
            if tax is not None: break
    for n in ebt_names:
        if n in income.index:
            ebt = _latest_non_na(income.loc[n])
            if ebt not in (None, 0): break
    if tax is None or ebt in (None, 0):
        return None
    return _bound(tax/ebt, 0.0, 0.40)


def _compute_kd(income, balance):
    if income is None or getattr(income, "empty", True) or balance is None or getattr(balance, "empty", True):
        return None
    # Intereses
    interest = None
    for n in ["Interest Expense","Interest Expense Non Operating","Net Interest Income"]:
        if n in income.index:
            v = _latest_non_na(income.loc[n])
            if v is not None:
                interest = abs(v); break
    if interest is None:
        return None
    # Deuda promedio
    debt_series = None
    for n in ["Total Debt","Long Term Debt","Long Term Debt And Capital Lease Obligation","Short Long Term Debt"]:
        if n in balance.index:
            s = _to_numeric_series(balance.loc[n]).dropna()
            if not s.empty:
                debt_series = s.astype(float); break
    if debt_series is None or debt_series.empty:
        return None
    avg_debt = float(debt_series.iloc[:2].mean()) if len(debt_series) >= 2 else float(debt_series.iloc[0])
    if avg_debt <= 0:
        return None
    return _bound(interest/avg_debt, 0.0, 0.20)


def _get_market_cap(t):
    # 1) info.marketCap
    try:
        info = t.info or {}
        mc = _finite_or_none(info.get("marketCap"))
        if mc is not None:
            return mc
    except Exception:
        pass
    # 2) Precio * acciones
    try:
        hist = t.history(period="1d")
        if hist is not None and not hist.empty:
            price = _as_float(hist["Close"].iloc[-1])
            sh = t.get_shares_full(start="2000-01-01")
            if sh is not None and not sh.empty:
                # intentar distintas columnas posibles
                for col in ["Shares Outstanding","Basic Shares Outstanding","Basic Average Shares"]:
                    if col in sh.columns:
                        shares = _as_float(sh[col].iloc[-1])
                        if np.isfinite(price) and np.isfinite(shares):
                            return price * shares
    except Exception:
        return None
    return None


def _get_beta(t):
    try:
        info = t.info or {}
        for k in ("beta","beta3Year","beta5Year"):
            v = _finite_or_none(info.get(k))
            if v is not None:
                return v
    except Exception:
        pass
    return None


def calcular_wacc_yahoo(ticker, mrp=0.055, rf=None):
    t = yf.Ticker(ticker)
    try: income = t.financials
    except Exception: income = None
    try: balance = t.balance_sheet
    except Exception: balance = None

    E = _get_market_cap(t)
    D = None
    if balance is not None and not balance.empty and "Total Debt" in balance.index:
        s = _to_numeric_series(balance.loc["Total Debt"]).dropna()
        if not s.empty:
            D = _finite_or_none(s.iloc[0])
    if D is None:
        D = 0.0

    beta = _get_beta(t)
    rf_ = rf if rf is not None else _fetch_risk_free_rate()
    kd = _compute_kd(income, balance)
    tax = _compute_tax_rate(income)

    notes = []
    # Defaults si faltan / no finitos
    if not np.isfinite(_as_float(rf_)):
        rf_, notes = 0.04, notes+["RF ^TNX no disponible; 4.0% por defecto"]
    if not np.isfinite(_as_float(beta)):
        beta, notes = 1.0, notes+["β no disponible; 1.0 por defecto"]
    if not np.isfinite(_as_float(kd)):
        kd, notes = 0.06, notes+["Kd no disponible; 6.0% por defecto"]
    if not np.isfinite(_as_float(tax)):
        tax, notes = 0.25, notes+["T no disponible; 25% por defecto"]
    if not np.isfinite(_as_float(E)):
        return {"Ticker": ticker, "WACC": np.nan, "WACC_Notas": "Sin Market Cap (no numérico)"}

    # Saneos finales
    E = _bound(E, 0.0, None)
    D = _bound(D, 0.0, None)
    beta = _bound(beta, -5.0, 5.0)
    rf_ = _bound(rf_, 0.0, 0.20)
    kd = _bound(kd, 0.0, 0.30)
    tax = _bound(tax, 0.0, 0.50)

    V = E + D if np.isfinite(E) and np.isfinite(D) else np.nan
    if not np.isfinite(V) or V <= 0:
        return {"Ticker": ticker, "WACC": np.nan, "WACC_Notas": "E+D inválido"}

    ke = rf_ + beta*mrp
    wacc = (E/V)*ke + (D/V)*kd*(1.0 - tax)

    return {"Ticker": ticker, "WACC": float(wacc), "WACC_Notas": "; ".join(notes)}

# ========= Helpers ROIC (con chequeos) =========
def _safe(series, key):
    try:
        return _as_float(series.get(key, np.nan))
    except Exception:
        return np.nan


def calcular_roic_yahoo(ticker):
    t = yf.Ticker(ticker)
    try: is_ = t.financials
    except Exception: is_ = pd.DataFrame()
    try: bs_ = t.balance_sheet
    except Exception: bs_ = pd.DataFrame()

    if is_.empty or bs_.empty:
        return {"Ticker": ticker, "ROIC": np.nan, "ROIC_Notas": "sin datos suficientes"}

    icols = list(is_.columns)[:2] if len(is_.columns) >= 2 else list(is_.columns)
    bcols = list(bs_.columns)[:2] if len(bs_.columns) >= 2 else list(bs_.columns)
    if not icols or not bcols:
        return {"Ticker": ticker, "ROIC": np.nan, "ROIC_Notas": "sin periodos suficientes"}

    if "Ebit" in is_.index:
        ebit = _safe(is_.loc["Ebit"], icols[0])
    elif "Operating Income" in is_.index:
        ebit = _safe(is_.loc["Operating Income"], icols[0])
    else:
        ebit = np.nan

    income_bt = _as_float(is_.loc["Income Before Tax", icols[0]]) if "Income Before Tax" in is_.index else np.nan
    tax_exp   = _as_float(is_.loc["Income Tax Expense", icols[0]]) if "Income Tax Expense" in is_.index else np.nan
    try:
        eff_tax = np.nan if (pd.isna(income_bt) or income_bt == 0 or pd.isna(tax_exp)) else tax_exp / income_bt
        eff_tax = _bound(eff_tax, 0.0, 0.35)
    except Exception:
        eff_tax = np.nan
    if np.isnan(eff_tax):
        eff_tax = 0.25

    nopat = np.nan if np.isnan(ebit) else float(ebit) * (1 - eff_tax)

    def invested_cap_at(col):
        total_debt = 0.0
        for k in ['Total Debt','Short Long Term Debt','Long Term Debt','Short Term Debt','Current Debt','Long Term Debt Noncurrent']:
            if k in bs_.index:
                val = _as_float(bs_.loc[k, col])
                if np.isfinite(val):
                    total_debt += val
        equity = _as_float(bs_.loc['Total Stockholder Equity', col]) if 'Total Stockholder Equity' in bs_.index else np.nan
        equity = 0.0 if not np.isfinite(equity) else equity
        cash   = _as_float(bs_.loc['Cash And Cash Equivalents', col]) if 'Cash And Cash Equivalents' in bs_.index else np.nan
        cash   = 0.0 if not np.isfinite(cash) else cash
        val = total_debt + equity - cash
        return max(0.0, val) if np.isfinite(val) else np.nan

    ic_vals = [invested_cap_at(c) for c in bcols]
    ic_vals = [v for v in ic_vals if np.isfinite(v)]
    ic_avg  = np.nan if len(ic_vals)==0 else float(np.mean(ic_vals))

    roic = np.nan
    if np.isfinite(ic_avg) and ic_avg != 0 and np.isfinite(nopat):
        roic = float(nopat) / float(ic_avg)

    return {"Ticker": ticker, "ROIC": roic, "ROIC_Notas": "" if np.isfinite(roic) else "ROIC no estimable"}


def _roe_quarterly_std_yahoo(ticker, max_q=8):
    """
    Desvío estándar del ROE trimestral (~2 años).
    ROE_q ≈ Net Income / Total Stockholder Equity del mismo corte trimestral.
    """
    t = yf.Ticker(ticker)
    try: q_is = t.quarterly_financials
    except Exception: q_is = pd.DataFrame()
    try: q_bs = t.quarterly_balance_sheet
    except Exception: q_bs = pd.DataFrame()

    if q_is.empty or q_bs.empty: return np.nan
    cols = list(q_is.columns)[:max_q]
    if not cols: return np.nan

    roes = []
    for c in cols:
        try:
            ni = _as_float(q_is.loc["Net Income", c]) if "Net Income" in q_is.index else np.nan
            eq = _as_float(q_bs.loc["Total Stockholder Equity", c]) if ("Total Stockholder Equity" in q_bs.index and c in q_bs.columns) else np.nan
            if np.isfinite(ni) and np.isfinite(eq) and eq != 0:
                roes.append(float(ni)/float(eq))
        except Exception:
            continue

    if len(roes) < 2: return np.nan
    return float(np.std(roes, ddof=1))


def exportar_lista_empresas(tickers_comparables):
    campos = {
        "Market Cap": "marketCap",
        "P/E": "trailingPE",
        "EPS": "trailingEps",
        "EPS Growth": "earningsQuarterlyGrowth",
        "Debt to Equity": "debtToEquity",
        "ROE": "returnOnEquity",
        "Dividend Yield": "dividendYield"
    }

    # descargar S&P500 (Adj Close) una vez para todos los tickers
    sp500_series = None
    try:
        sp_df = yf.download("^GSPC", start="2010-01-01", progress=False)
        if sp_df is not None and not sp_df.empty:
            sp500_series = sp_df["Adj Close"] if "Adj Close" in sp_df.columns else sp_df["Close"]
    except Exception:
        sp500_series = None

    filas = []
    for tick in tickers_comparables:
        try:
            info = yf.Ticker(tick).info
            row = {"Ticker": tick}
            for nombre, clave in campos.items():
                row[nombre] = _as_float(info.get(clave, np.nan))

            # company name (longName o shortName)
            company_name = info.get("longName") or info.get("shortName") or tick
            row["Company"] = company_name

            # === Valor en libros / Precio sobre valor en libros ===
            # bookValue provisto por Yahoo
            book_val = _as_float(info.get("bookValue", np.nan))
            # precio preferido (currentPrice o regularMarketPrice), fallback a histórico
            price = _as_float(info.get("currentPrice", info.get("regularMarketPrice", np.nan)))
            if not np.isfinite(price):
                try:
                    hist = yf.Ticker(tick).history(period="5d", actions=False)
                    if hist is not None and not hist.empty:
                        price = _as_float(hist["Close"].iloc[-1])
                except Exception:
                    price = np.nan
            # priceToBook directo si existe, sino cálculo fallback price/book_val
            pb = _as_float(info.get("priceToBook", np.nan))
            if not np.isfinite(pb) and np.isfinite(price) and np.isfinite(book_val) and book_val != 0:
                pb = float(price) / float(book_val)
            row["Book Value"] = book_val
            row["Price/Book"] = pb

            # === Métricas de dividendos ===
            # Monto anual del dividendo (por acción) desde Yahoo si está disponible
            annual_div = _as_float(info.get("dividendRate", np.nan))

            # Escudo de dividendos = dividendo anual / precio
            div_shield = np.nan
            if np.isfinite(annual_div) and np.isfinite(price) and price != 0:
                div_shield = float(annual_div) / float(price)

            # Crecimiento de dividendos a 5 años: calcular a partir del historial de dividendos (yfinance) si es posible
            div_growth_5y = np.nan
            try:
                divs = yf.Ticker(tick).dividends
                if isinstance(divs, (pd.Series,)) and not divs.empty:
                    # Make sure datetime index
                    divs = divs.sort_index()
                    yearly = divs.resample('Y').sum()
                    # take last 6 years to get 5 year changes
                    yearly = yearly[-6:]
                    if len(yearly.dropna()) >= 2:
                        yoy = yearly.pct_change().dropna()
                        last_n = yoy[-5:]
                        if len(last_n) > 0:
                            div_growth_5y = float(last_n.mean())
            except Exception:
                div_growth_5y = np.nan

            # Correlación con S&P500 desde 2010 (usa sp500_series si está disponible)
            corr_sp = np.nan
            try:
                if sp500_series is not None:
                    tdf = yf.download(tick, start="2010-01-01", progress=False)
                    if tdf is not None and not tdf.empty:
                        t_adj = tdf["Adj Close"] if "Adj Close" in tdf.columns else tdf["Close"]
                        sp_ret = sp500_series.pct_change().dropna()
                        t_ret = t_adj.pct_change().dropna()
                        common = pd.concat([t_ret, sp_ret], axis=1, join='inner')
                        if common.shape[0] >= 30:
                            corr_val = common.corr().iloc[0,1]
                            if np.isfinite(_as_float(corr_val)):
                                corr_sp = float(corr_val)
            except Exception:
                corr_sp = np.nan

            # Adjuntar las métricas de dividendos calculadas a la fila.
            row["Dividend Rate"] = annual_div
            row["Dividend Shield"] = div_shield
            row["Dividend Growth 5Y"] = div_growth_5y
            row["Corr S&P500"] = corr_sp

            # Continuar con los cálculos previamente existentes.
            wacc = calcular_wacc_yahoo(tick)
            roic = calcular_roic_yahoo(tick)
            row["WACC (interno)"] = _as_float(wacc.get("WACC", np.nan))
            row["ROIC (interno)"] = _as_float(roic.get("ROIC", np.nan))

            filas.append(row)
        except Exception as e:
            print(f"Error en {tick}: {e}")
            continue

    df = pd.DataFrame(filas)

    # Imputación para variables base
    cols_num = list(campos.keys())
    for col in cols_num:
        if col in df.columns:
            media = df[col].mean(skipna=True)
            df[col] = df[col].fillna(media)
        else:
            df[col] = np.nan

    # Asegurar que existan las columnas relacionadas con dividendos y hacer imputación sensata
    if "Dividend Rate" not in df.columns:
        df["Dividend Rate"] = np.nan
    if "Dividend Shield" not in df.columns:
        df["Dividend Shield"] = np.nan
    if "Dividend Growth 5Y" not in df.columns:
        df["Dividend Growth 5Y"] = np.nan

    # Asegurar que exista Corr S&P 500.
    if "Corr S&P500" not in df.columns:
        df["Corr S&P500"] = np.nan

    # Completar Dividend Shield con la mediana (para que no penalice en exceso los NaN).
    med_shield = float(np.nanmedian(pd.to_numeric(df["Dividend Shield"], errors="coerce"))) if df["Dividend Shield"].notna().any() else 0.0
    if not np.isfinite(med_shield):
        med_shield = 0.0
    df["Dividend Shield"] = pd.to_numeric(df["Dividend Shield"], errors="coerce").fillna(med_shield)

    # Crecimiento de dividendos: completar los NaN con 0 (sin crecimiento / sin dividendos).
    df["Dividend Growth 5Y"] = pd.to_numeric(df["Dividend Growth 5Y"], errors="coerce").fillna(0.0)

    # Correlación con el S&P 500: completar los NaN con 0 (no se puede calcular la correlación).
    df["Corr S&P500"] = pd.to_numeric(df["Corr S&P500"], errors="coerce").fillna(0.0)

    # Relación ROIC/WACC (winsorización + imputación por la mediana).
    ratio = df["ROIC (interno)"] / df["WACC (interno)"]
    ratio = pd.to_numeric(ratio, errors="coerce").replace([np.inf, -np.inf], np.nan)
    finite_ratio = ratio[np.isfinite(ratio)]
    p95 = float(np.nanpercentile(finite_ratio, 95)) if len(finite_ratio) > 0 else np.nan
    if np.isnan(p95) or p95 <= 0: p95 = 5.0
    ratio = ratio.clip(lower=0.0, upper=p95)
    med = float(np.nanmedian(ratio)) if np.isfinite(ratio).any() else 1.0
    df["Roic/WACC"] = ratio.fillna(med)

    # Asegurar que exista la columna Company y completarla con el Ticker si falta.
    if "Company" not in df.columns:
        df["Company"] = df["Ticker"].astype(str)
    else:
        df["Company"] = df["Company"].fillna(df["Ticker"].astype(str))

    return df

Dashboard Interactivo

In [22]:
""" Primero se define los tickers con los cuales vas a comparar"""
if __name__ == "__main__":
    tickers = [
        "UNH", "FI", "UPS", "IFF", "MSFT", "PDD", "DIS", "ASML", "WBD", "PEP", "PG", "AMD", "AMAT",
        "XPEV", "NIO", "BIDU", "PFE", "LEN", "JOE", "META", "AER", "BKNG", "WIX", "ORLA", "CVS", "KHC", "COKE", "NVDA",
        "AMZN", "GOOGL", "TSLA", "JPM", "V", "MA", "HD", "BAC", "WMT", "OLN", "DECK", "FIVE", "GRBK", "FLR", "KD", "MOH",
        "SLM", "RIG", "IONQ", "RGTI", "QBTS", "QUBT", "ARQQ"
    ]
    # Descargar métricas y construir DataFrame base
    df_emp = exportar_lista_empresas(tickers)
    resultado = df_emp.copy()

    # Reordenar columnas para mostrar primero Ticker y Company
    cols_order = [c for c in resultado.columns if c not in ("Ticker", "Company")]
    resultado = resultado[["Ticker", "Company"] + cols_order]

    # ipywidgets para interacción
    try:
        import ipywidgets as widgets
        from IPython.display import display, clear_output
        interactive_available = True
    except Exception:
        interactive_available = False

    # Formatos para Styler
    format_map = {
        "Market Cap": "{:.0f}",
        "P/E": "{:.2f}",
        "EPS": "{:.2f}",
        "EPS Growth": "{:.2f}",
        "Debt to Equity": "{:.2f}",
        "ROE": "{:.2f}",
        "Roic/WACC": "{:.2f}",
        "WACC (interno)": "{:.3f}",
        "ROIC (interno)": "{:.3f}",
        "Dividend Shield": "{:.3%}",
        "Dividend Growth 5Y": "{:.2%}",
        "Corr S&P500": "{:.3f}",
        "Price/Book": "{:.2f}",
        "Book Value": "{:.0f}"
    }

    # Estilos para la tabla.
    common_table_styles = [
        {"selector": "thead th.row_heading", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#ffffff"), ("z-index", "4")]},
        {"selector": "thead th.col_heading", "props": [("position", "sticky"), ("top", "0"), ("background-color", "#f7fbff"), ("z-index", "3"), ("border-bottom", "2px solid #cfcfcf"), ("text-align", "left")]},
        {"selector": "tbody th", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#ffffff"), ("z-index", "2"), ("font-weight", "bold")]},
        {"selector": "tbody td", "props": [("white-space", "nowrap"), ("border-bottom", "1px solid #e8e8e8"), ("padding", "6px 8px")]},
        {"selector": "table", "props": [("border-collapse", "separate"), ("border-spacing", "0 4px")]}
    ]

    if not interactive_available:
        # Versión estática.
        styled = resultado.style.format(format_map) \
            .set_table_attributes('style="display:block; max-width:100%; overflow:auto;"') \
            .set_table_styles(common_table_styles)
        # destacar columna Company
        if "Company" in resultado.columns:
            styled = styled.set_properties(subset=["Company"], **{"color": "#2B8CFF", "font-weight": "bold"})
        # gradiente para índice (si existe)
        if "Índice Compuesto" in resultado.columns:
            styled = styled.background_gradient(cmap="Greens", subset=["Índice Compuesto"]) \
                .set_caption("Ranking fundamental Modelo 2")
        else:
            styled = styled.set_caption("Datos fundamentales (sin índice compuesto)")
        display(styled)
    else:
        # Versión interactiva con widgets
        cols = list(resultado.columns)
        default_col = "Índice Compuesto" if "Índice Compuesto" in cols else (cols[0] if cols else None)
        col_dd = widgets.Dropdown(options=cols, value=default_col, description='Ordenar por:')
        asc_toggle = widgets.ToggleButtons(options=[('Desc', False), ('Asc', True)], description='Orden:')
        max_rows = max(5, min(500, len(resultado)))
        rows_slider = widgets.IntSlider(value=min(50, max_rows), min=5, max=max_rows, step=1, description='Top N:')

        out = widgets.Output()

        def render(order_by, ascending, top_n):
            with out:
                clear_output()
                try:
                    df_sorted = resultado.sort_values(order_by, ascending=ascending)
                except Exception:
                    # Si la columna no es ordenable directamente, intentar convertir a num
                    df_sorted = resultado.copy()
                    try:
                        df_sorted[order_by] = pd.to_numeric(df_sorted[order_by], errors='coerce')
                        df_sorted = df_sorted.sort_values(order_by, ascending=ascending)
                    except Exception:
                        pass

                if top_n is not None and top_n > 0:
                    df_display = df_sorted.head(top_n)
                else:
                    df_display = df_sorted

                styled = df_display.style.format(format_map) \
                    .set_table_attributes('style="display:block; max-width:100%; overflow:auto;"') \
                    .set_table_styles(common_table_styles)
                if "Company" in df_display.columns:
                    styled = styled.set_properties(subset=["Company"], **{"color": "#2B8CFF", "font-weight": "bold"})
                # Aplicar gradiente solo si la columna existe
                if "Índice Compuesto" in df_display.columns:
                    styled = styled.background_gradient(cmap="Greens", subset=["Índice Compuesto"]) \
                        .set_caption(f"Ranking fundamental Modelo 2 | Ordenado por {order_by} ({'asc' if ascending else 'desc'}) | Top {top_n}")
                else:
                    styled = styled.set_caption(f"Datos fundamentales | Ordenado por {order_by} ({'asc' if ascending else 'desc'}) | Top {top_n}")
                display(styled)

        # Controles interactivos
        ui = widgets.HBox([col_dd, asc_toggle, rows_slider])
        render(col_dd.value, asc_toggle.value, rows_slider.value)  # initial render
        def _on_change(change):
            render(col_dd.value, asc_toggle.value, rows_slider.value)
        col_dd.observe(_on_change, names='value')
        asc_toggle.observe(_on_change, names='value')
        rows_slider.observe(_on_change, names='value')

        display(ui)
        display(out)

C:\Users\nicon\AppData\Local\Temp\ipykernel_2464\3297149965.py:267: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp_df = yf.download("^GSPC", start="2010-01-01", progress=False)
C:\Users\nicon\AppData\Local\Temp\ipykernel_2464\3297149965.py:320: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly = divs.resample('Y').sum()
C:\Users\nicon\AppData\Local\Temp\ipykernel_2464\3297149965.py:335: FutureWarning: YF.download() has changed argument auto_adjust default to True
  tdf = yf.download(tick, start="2010-01-01", progress=False)
C:\Users\nicon\AppData\Local\Temp\ipykernel_2464\3297149965.py:320: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly = divs.resample('Y').sum()
C:\Users\nicon\AppData\Local\Temp\ipykernel_2464\3297149965.py:335: FutureWarning: YF.download() has changed argument auto_adjust default to True
  tdf = yf.download(tick, s

Output()

Visualización desde GitHub.

In [23]:
# Vista simple: ordenar y mostrar los datos básicos en la notebook
df_simple = resultado.copy()
df_simple = df_simple.sort_values(by="Ticker").reset_index(drop=True)
display(df_simple.head(100))

,Ticker,Company,Market Cap,P/E,EPS,EPS Growth,Debt to Equity,ROE,Dividend Yield,Book Value,Price/Book,Dividend Rate,Dividend Shield,Dividend Growth 5Y,Corr S&P500,WACC (interno),ROIC (interno),Roic/WACC
0,AER,AerCap Holdings N.V.,2.488134e+10,6.663801,20.94,2.242000,243.287000,0.21712,0.770000,109.224000,1.277558,1.08,0.007740,0.080000,0.567485,0.051691,0.017637,0.341195
1,AMAT,"Applied Materials, Inc.",2.064977e+11,29.931870,8.66,0.096000,34.533000,0.35508,0.710000,25.744000,10.068754,1.84,0.007098,0.155720,0.698913,0.130499,NaN,2.210831
2,AMD,"Advanced Micro Devices, Inc.",3.431586e+11,109.781250,1.92,0.612000,6.366000,0.05317,2.015862,37.340000,5.644885,NaN,0.013441,0.000000,0.520836,0.147160,1.858076,12.626207
3,AMZN,"Amazon.com, Inc.",2.418020e+12,31.947740,7.08,0.382000,43.405000,0.24327,2.015862,34.587000,6.539740,NaN,0.013441,0.000000,0.608212,0.112173,0.456714,4.071519
4,ARQQ,Arqit Quantum Inc.,4.437302e+08,41.900118,-2.57,0.813706,2.640000,-1.81436,2.015862,1.781000,15.918024,NaN,0.013441,0.000000,0.180936,0.172811,NaN,2.210831
5,ASML,ASML Holding N.V.,4.195294e+11,38.165607,28.32,0.023000,14.240000,0.53852,0.680000,57.598656,18.765194,7.37,0.006819,0.234879,0.672640,0.115730,5.992429,34.861496
6,BAC,Bank of America Corporation,4.084191e+11,15.065574,3.66,0.228000,88.910149,0.09871,2.030000,37.951000,1.452926,1.12,0.020312,0.084524,0.702511,0.145582,NaN,2.210831
7,BIDU,"Baidu, Inc.",4.357919e+10,11.333636,11.03,0.813706,33.810000,0.03076,2.015862,110.564080,1.130657,NaN,0.013441,0.000000,0.453325,0.042040,0.122077,2.903810
8,BKNG,Booking Holdings Inc.,1.718255e+11,34.482212,153.75,0.092000,88.910149,0.11126,0.720000,-146.596000,-36.164974,38.40,0.007243,0.097143,0.601617,0.106810,0.329673,3.086535
9,COKE,"Coca-Cola Consolidated, Inc.",1.104326e+10,23.649073,7.01,0.231000,117.146000,0.41924,0.600000,19.039000,8.707390,1.00,0.006032,1.300000,0.381682,0.065136,0.423922,6.508295
